# **Lab 1: Matrix algorithms**
**Helmer Nylén**

# **Abstract**

In this lab we implement scalar, matrix-vector and matrix-matrix products as well as Euclidean norm and distance. We verify our implementation against `numpy` equivalents and conclude that the implementation is correct.

#**About the code**

In [1]:
"""This program is a lab report in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Helmer Nylén (helmern@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

In this report we aim to implement a number of elementary vector and matrix operations, using our own data structures rather than those provided by `numpy`. These are: the scalar product $c = (x, y)$, the matrix-vector product $y = Ax$, the matrix-matrix product $C = AB$, the Euclidean norm $c = ||x||$ and the Euclidean distance $c = ||x - y||$.




<!-- Give a short description of the problem investigated in the report, and provide some background information so that the reader can understand the context. 

Briefly describe what method you have chosen to solve the problem, and justify why you selected that method. 

Here you can express mathematics through Latex syntax, and use hyperlinks for references.

[Hyperlink to DD2363 course website.](https://kth.instructure.com/courses/7500)

$
{\displaystyle \frac{\partial u}{\partial t}} + u\cdot \nabla u +\nabla p = f, \quad \nabla \cdot u=0$ -->



# **Methods**

We implement the scalar product and Euclidean norm as in example 1.6 in chapter 1.4 of the lecture notes. For vectors $x, y \in \mathbb{R}^n$:
$$(x, y) = x \cdot y = x_1 y_1 + \ldots + x_n y_n,$$
$$||x|| = ||x||_2 = (x, x)^\frac{1}{2}.$$
It follows that the Euclidean distance can be implemented as
$$d(x, y) = ||x-y|| = (x-y, x-y)^\frac{1}{2}.$$

The matrix-vector product is implemented as in the beginning of chapter 2.2. Let $A$ be an $m \times n$ matrix, $x$ an $n$-length vector and $y = Ax$. Then
$$y_i = \sum_{j=1}^n a_{ij} x_j, \qquad i = 1, \ldots, m.$$
Likewise, the matrix-matrix product is implemented as in eq. 2.2 chapter 2.3. Let $A \in \mathbb{R}^{m \times l}, C \in \mathbb{R}^{l \times n}$. Then $B = AC \in \mathbb{R}^{m \times n}$ is defined by
$$b_{ij} = \sum_{k=1}^{l} a_{ik}c_{kj}.$$

In the code  the vectors and matrices will be zero-indexed to make implementation easier.

### Code for the `Vector` class

In [0]:
class Vector:
    def __init__(self, *elems, row = False):
        self.elems = tuple(elems)
        self.row = row

    def __getitem__(self, indices):
        return self.elems[indices]
    
    def __len__(self):
        return len(self.elems)
    
    def __str__(self):
        if self.row:
            return str(self.elems)
        else:
            return str(self.elems) + "^T"
    
    def __add__(self, other):
        assert len(self) == len(other) and self.row == other.row
        return Vector(*(self[i] + other[i] for i in range(len(self))), row=self.row)
    
    def __sub__(self, other):
        assert len(self) == len(other) and self.row == other.row
        return Vector(*(self[i] - other[i] for i in range(len(self))), row=self.row)
    
    def __mul__(self, other):
        # Scalar product
        if type(other) is Vector:
            assert len(self) == len(other) and self.row == other.row
            return sum(self[i] * other[i] for i in range(len(self)))

        elif type(other) is Matrix:
            raise TypeError("Please use matrix multiplication")

        else:
            return Vector(*(i * other for i in self.elems), row = self.row)
    
    def __matmul__(self, other):
        return self.to_matrix() @ other
    
    def __iter__(self):
        for i in self.elems:
            yield i

    def __eq__(self, other):
        if type(other) is not Vector or len(self) != len(other):
            return False
        for i in range(len(self)):
            if self[i] != other[i]:
                return False
        return True

    def __abs__(self):
        # Euclidean norm
        return (self * self) ** (1/2)
    
    def distance(self, other):
        # Euclidean distance
        return abs(self - other)

    def to_matrix(self):
        if self.row:
            return Matrix(self,)
        else:
            return Matrix(*((x,) for x in self))
    
    def T(self):
        return Vector(*self.elems, row = not self.row)

### Code for the `Matrix` class

In [0]:
class Matrix:
    def __init__(self, *rows):
        if len(rows) > 0:
            width = len(rows[0])
            for row in rows:
                assert len(row) == width
            self.size = (len(rows), len(rows[0]))
        else:
            self.size = (0, 0)
        self.elems = tuple(tuple(row) for row in rows)
    
    def __len__(self):
        return len(self.elems)
    
    def __getitem__(self, indices):
        return self.elems[indices[0]][indices[1]]

    def __matmul__(self, other):
        # Matrix-matrix product
        if type(other) is Matrix:
            assert self.size[1] == other.size[0]
            res = [[0] * other.size[1] for x in range(self.size[0])]
            for i in range(self.size[0]):
                for j in range(other.size[1]):
                    res[i][j] = sum(self[i, k] * other[k, j] for k in range(self.size[1]))
            return Matrix(*res)
            
        # Matrix-vector product
        elif type(other) is Vector:
            assert self.size[1] == len(other) and other.row == False
            res = [0] * self.size[0]
            for i in range(self.size[0]):
                res[i] = sum(self[i, j] * other[j] for j in range(self.size[1]))

            return Vector(*res)

    def __str__(self):
        if self.size == (0, 0):
            return "(())"
        return "(" + ",\n".join(str(row) for row in self.elems) + ")"
    
    def __eq__(self, other):
        if type(other) is not Matrix or self.size != other.size:
            return False
        for i in range(self.size[0]):
            for j in range(self.size[1]):
                if self[i, j] != other[i, j]:
                    return False
        return True

### Tests

To verify the correctness of our implementation we assume that the `numpy` equivalents are correct and compare our results to those. We also perform a number of sanity checks making sure that undefined behaviour (such as calculating the scalar product of vectors of different lengths) is disallowed.

We perform the tests on random integer or float data to demonstrate that the implementation works not only for hard-coded data but in general.

In [0]:
from random import randint, uniform
class Tests:
    @staticmethod
    def randvec(n, randfunc = randint):
        return Vector(*(randfunc(-10, 10) for x in range(n)))
    @staticmethod
    def randmat(n, m, randfunc = randint):
        return Matrix(*[[randfunc(-10, 10) for x in range(m)] for y in range(n)])
    @staticmethod
    def vecmul(randfunc = randint, verbose = False):
        n = randint(1, 5)
        a = Tests.randvec(n, randfunc)
        b = Tests.randvec(n, randfunc)
        c = a * b
        if verbose:
          print("a:", a)
          print("b:", b)
          print("a * b:", c)
        assert a.T() * b.T() == c
        assert np.isclose(c, np.array(a.elems).dot(np.array(b.elems)))

        try:
            b *= Tests.randvec(n + 1, randfunc)
        except AssertionError:
            if verbose:
              print("As expected, cannot multiply vectors of different lengths")
        else:
            assert False

    @staticmethod
    def matvecmul(randfunc = randint, verbose = False):
        n, m, p = randint(1, 10), randint(1, 10), randint(1, 10)
        x = Tests.randvec(m, randfunc)
        A = Tests.randmat(n, m, randfunc)
        y = A @ x
        B = Tests.randmat(p, n, randfunc)
        z = B @ y
        if verbose:
          print("x:", x)
          print("A:", A)
          print("A @ x:", y)
          print("B:", B)
          print("B @ A @ x:", z)
        
        try:
            q = A @ Tests.randvec(m + 1, randfunc)
        except AssertionError:
            if verbose:
              print("As expected, cannot multiply an (n x m)-matrix with an (m+1)-vector")
        else:
            assert False
        
        assert np.allclose(np.array(z), np.array(B.elems) @ np.array(A.elems) @ np.array(x.elems))

        a = Tests.randvec(m, randfunc)
        b = Tests.randvec(m, randfunc)
        assert a * b == (a.T().to_matrix() @ b)[0]
    
    @staticmethod
    def matmul(randfunc = randint, verbose = False):
        n, m, p = randint(1, 10), randint(1, 10), randint(1, 10)
        x = Tests.randvec(m, randfunc)
        A = Tests.randmat(n, m, randfunc)
        B = Tests.randmat(p, n, randfunc)
        try:
            C = B @ Tests.randmat(n + 1, m, randfunc)
        except AssertionError:
            if verbose:
                print("As expected, matrix dimensions must agree")
        else:
            assert False
        C = B @ A
        z = C @ x
        if verbose:
          print("B @ A:", C)
        assert z == B @ (A @ x)
        assert np.allclose(np.array(C.elems), np.array(B.elems) @ np.array(A.elems))

    @staticmethod
    def norm(randfunc = randint, verbose = False):
        a = Tests.randvec(randint(1, 10), randfunc)
        if verbose:
          print("a:", a, "norm:", abs(a))
        assert np.isclose(abs(a), np.linalg.norm(np.array(a.elems)))

    @staticmethod
    def dist(randfunc = randint, verbose = False):
        n = randint(1, 10)
        a = Tests.randvec(n, randfunc)
        b = Tests.randvec(n, randfunc)
        if verbose:
          print("a:", a)
          print("b:", b)
          print("Distance:", a.distance(b))
        assert a.distance(b) == b.distance(a)
        assert a.distance(a) == 0
        assert np.isclose(a.distance(b), np.linalg.norm(np.array(a.elems) - np.array(b.elems)))

# **Results**

In [6]:
for randfunc in [randint, uniform]:
  for i in range(1000):
    Tests.vecmul(randfunc = randfunc)
    Tests.matvecmul(randfunc = randfunc)
    Tests.matmul()
    Tests.norm()
    Tests.dist()

print("All tests passed")

All tests passed


All the tests passed using both integer and float data.

# **Discussion**

As expected the tests all passed, leading us to believe that the implementation is correct.